In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
train_2016_df = pd.read_csv("../input/zillow-prize-1/train_2016_v2.csv", parse_dates=["transactiondate"], index_col='parcelid')
train_2017_df = pd.read_csv("../input/zillow-prize-1/train_2017.csv", parse_dates=["transactiondate"], index_col='parcelid')
prop_2016_df = pd.read_csv("../input/zillow-prize-1/properties_2016.csv", index_col='parcelid', low_memory=False)
prop_2017_df = pd.read_csv("../input/zillow-prize-1/properties_2017.csv", index_col='parcelid', low_memory=False)
sample_sub = pd.read_csv("../input/zillow-prize-1/sample_submission.csv")


# Merge datasets
train_2016_df = pd.merge(train_2016_df, prop_2016_df, on='parcelid', how='left')
train_2017_df = pd.merge(train_2017_df, prop_2017_df, on='parcelid', how='left')

X_full = pd.concat([train_2016_df, train_2017_df], axis=0)

In [ ]:
# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['logerror'], inplace=True)
y = X_full.logerror
X_full.drop(['logerror'], axis=1, inplace=True)

In [ ]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [ ]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

In [ ]:
# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

In [ ]:
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = prop_2016_df[my_cols].copy()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = XGBRegressor(n_estimators=138, learning_rate=0.05, random_state=0)

In [ ]:
# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

In [ ]:
def score_dataset(X_train, X_valid, y_train, y_valid, ne, lr):
    
    model = XGBRegressor(n_estimators=ne, learning_rate=lr, random_state=0)
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])
    clf.fit(X_train, y_train)
    preds = clf.predict(X_valid)
    
    return mean_absolute_error(y_valid, preds)


In [ ]:
sample_sub['parcelid'] = sample_sub['ParcelId']
sub = sample_sub.merge(prop_2016_df, on='parcelid', how='left')
sub.head()

In [ ]:
preds_full = clf.predict(X_test)

In [ ]:
sub['201610'] = preds_full
sub['201611'] = preds_full
sub['201612'] = preds_full
sub['201710'] = preds_full
sub['201711'] = preds_full
sub['201712'] = preds_full

In [ ]:
sub = sub[['ParcelId', '201610', '201611', '201612', '201710', '201711', '201712']]
sub.head()

In [ ]:
print('Writing csv ...')
sub.to_csv('xg_boost.csv', index=False, float_format='%.4f') # Thanks to @inversion